In [1]:
from __future__ import print_function
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import json
from pprint import pprint
import numpy as np
import csv

In [2]:
from candidate import Candidate

In [3]:
SECRETS_FILE = "authentication.json"
OFFICER_SPREADSHEET = "Candidate Evaluation--Officer (Responses)"
LEAD_SPREADSHEET = "Candidate Evaluation--Lead (Responses)"
LB_SPREADSHEET = "Candidate Evaluation--Leaderboard (Responses)"

In [4]:
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(SECRETS_FILE, scope)

In [5]:
client = gspread.authorize(creds)

In [6]:
officer_data = client.open(OFFICER_SPREADSHEET).sheet1.get_all_records()
lead_data = client.open(LEAD_SPREADSHEET).sheet1.get_all_records()
lb_data = client.open(LB_SPREADSHEET).sheet1.get_all_records()

In [7]:
CANDIDATES = ["Xuezheng Wang", "Chen Ling", "Shuyang Miao", "Shantao Yi", 
              "Cheng Gong", "Wanju Gu", "Zhixiong Li", "Zixin Zeng", 
              "Weiwen Hong", "Ziyan Cui", "Weihao Zeng"]

In [8]:
DEPT = ["Strategy", "Technology", "Internal Affairs", "Career Development", "Inspiration", 
        "Business Development", "Meida", "New Dept./Committee"]

In [9]:
def get_c_d(response):
    return "{}-{}".format(response["Election Department"], response["Candidate's Name"])

In [10]:
def construct_c_d_list(officer_data, lead_data, lb_data):
    c_d_set = set()
    for r in officer_data:
        c_d_set.add(get_c_d(r))
    for r in lead_data:
        c_d_set.add(get_c_d(r))
    for r in lb_data:
        c_d_set.add(get_c_d(r))
    
    return list(c_d_set)

In [11]:
c_d_list = construct_c_d_list(officer_data, lead_data, lb_data)

In [12]:
def initializa_candidates(c_d_list):
    candidate_dict = {}
    for k in c_d_list:
        dept = k.split('-')[0]
        name = k.split('-')[1]
        candidate_dict[k] = Candidate(name, dept)
    return candidate_dict

In [13]:
candidate_dict = initializa_candidates(c_d_list)

In [14]:
for d in officer_data:
    name = d["Candidate's Name"]
    dept = d["Election Department"]
    candidate = candidate_dict["{}-{}".format(dept, name)]
    candidate.add_officer_score(d)

In [15]:
for d in lead_data:
    name = d["Candidate's Name"]
    dept = d["Election Department"]
    candidate = candidate_dict["{}-{}".format(dept, name)]
    candidate.add_lead_score(d)

In [16]:
for d in lb_data:
    name = d["Candidate's Name"]
    dept = d["Election Department"]
    candidate = candidate_dict["{}-{}".format(dept, name)]
    candidate.add_lb_score(d)

In [17]:
results = {}
for k,v in candidate_dict.items():
    v.calculate_scores()
    results[k] = v.score

In [18]:
result_list = []
for k,v in results.items():
    department = k.split('-')[0]
    name = k.split('-')[1]
    score = v
    result_list.append({
        "department": department,
        "name": name,
        "score": score,
    })

In [19]:
result_list = sorted(result_list, key=lambda elem: "%s %s" % (elem['department'], elem['score'])) 

In [20]:
with open("result.csv", 'wb') as f:
    keys = ['department', 'name', 'score']
    dict_writer = csv.DictWriter(f, keys)
    dict_writer.writeheader()
    dict_writer.writerows(result_list)

In [21]:
pprint(result_list)

[]
